In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [ ]:
data=pd.read_csv('/kaggle/input/craigslist-carstrucks-data/vehicles.csv')
data

In [ ]:
data.info()

# Preprocessing

In [ ]:
data.isna().sum()

In [ ]:
null_columns = data.columns[data.isna().mean()>0.25]
data = data.drop(null_columns,axis=1)

In [ ]:
data

In [ ]:
unneeded_cols = ['url','region_url','image_url','description','id','posting_date',]
data = data.drop(unneeded_cols,axis=1)

In [ ]:
data

In [ ]:
{column: len(data[column].unique()) for column in data.columns if data.dtypes[column] == 'object'}

In [ ]:
def onehot_encode(df,cols,prefixes):
    df=df.copy()
    for col,prefix in zip(cols,prefixes):
        dummies= pd.get_dummies(df[col],prefix=prefix)
        df= pd.concat([df,dummies],axis=1)
        df= df.drop(col,axis=1)
    return df

In [ ]:
df= onehot_encode(data,
                   ['region','manufacturer','fuel','title_status','transmission','type','state'],
                   ['reg','manf','fuel','status','trans','type','state'])

In [ ]:
df

In [ ]:
df = df.drop('model',axis=1)

In [ ]:
for column in df.columns:
    df[column] = df[column].fillna(df[column].mean())

In [ ]:
df.isna().sum().sum()

# Spliting and Scaling

In [ ]:
y = df.loc[:,'price']
X = df.drop('price',axis=1)

In [ ]:
scaler = StandardScaler()

X= scaler.fit_transform(X)

In [ ]:
X

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X,y, train_size = 0.7,random_state=34)

In [ ]:
lin_model = LinearRegression()
lin_model.fit(X_train,y_train)
lin_y_preds = lin_model.predict(X_test)

In [ ]:
lgb_model = lgb.LGBMRegressor(
            boosting_type = 'gbdt',
            num_leaves = 31,
            n_estimators = 100,
            reg_lambda=1.0)

lgb_model.fit(X_train,y_train)
lgb_y_preds = lgb_model.predict(X_test)

In [ ]:
lin_loss = np.sqrt(mean_squared_error(y_test,lin_y_preds))
lgb_loss = np.sqrt(mean_squared_error(y_test,lgb_y_preds))

In [ ]:
print('linear regression RMSE:', lin_loss)
print('Gradient boosted RMSE:', lgb_loss)